In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1258701,2021-05-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1258702,2021-05-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1258703,2021-05-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1258704,2021-05-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
33551,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33553,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33555,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33557,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33559,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
773914,2021-05-09,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804
773916,2021-05-10,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804
773918,2021-05-11,Arkansas,Arkansas,5001,2105,36.00,5000,Arkansas,AR,Arkansas,State,3017804
773920,2021-05-12,Arkansas,Arkansas,5001,2109,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1258701,2021-05-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1258702,2021-05-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1258703,2021-05-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1258704,2021-05-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-13') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
478,2021-05-13,Snohomish,Washington,37477,574.00,53061,WA,County,822083,4558.79,69.82
954,2021-05-13,Cook,Illinois,545041,10655.00,17031,IL,County,5150233,10582.84,206.88
1429,2021-05-13,Orange,California,270983,5026.00,6059,CA,County,3175692,8533.04,158.26
1903,2021-05-13,Maricopa,Arizona,543198,9940.00,4013,AZ,County,4485414,12110.32,221.61
2377,2021-05-13,Los Angeles,California,1237120,24067.00,6037,CA,County,10039107,12323.01,239.73
...,...,...,...,...,...,...,...,...,...,...,...
1257977,2021-05-13,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08
1258190,2021-05-13,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1258372,2021-05-13,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1258550,2021-05-13,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
478,2021-05-13,Snohomish,Washington,37477,574.00,53061,WA,County,822083,4558.79,69.82,69.82,4558.79
954,2021-05-13,Cook,Illinois,545041,10655.00,17031,IL,County,5150233,10582.84,206.88,206.88,10582.84
1429,2021-05-13,Orange,California,270983,5026.00,6059,CA,County,3175692,8533.04,158.26,158.26,8533.04
1903,2021-05-13,Maricopa,Arizona,543198,9940.00,4013,AZ,County,4485414,12110.32,221.61,221.61,12110.32
2377,2021-05-13,Los Angeles,California,1237120,24067.00,6037,CA,County,10039107,12323.01,239.73,239.73,12323.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257977,2021-05-13,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08,75.08,2402.40
1258190,2021-05-13,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1258372,2021-05-13,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1258550,2021-05-13,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
809265,2021-05-13,Hale,Alabama,2232,77.00,1065,AL,County,14651,15234.45,525.56,525.56,15234.45,1
880935,2021-05-13,Clarke,Alabama,3511,61.00,1025,AL,County,23622,14863.26,258.23,258.23,14863.26,2
633081,2021-05-13,Lowndes,Alabama,1390,53.00,1085,AL,County,9726,14291.59,544.93,544.93,14291.59,3
470946,2021-05-13,Franklin,Alabama,4262,82.00,1059,AL,County,31362,13589.69,261.46,261.46,13589.69,4
561292,2021-05-13,Etowah,Alabama,13861,353.00,1055,AL,County,102268,13553.60,345.17,345.17,13553.60,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192641,2021-05-13,Platte,Wyoming,631,11.00,56031,WY,County,8393,7518.17,131.06,131.06,7518.17,19
772277,2021-05-13,Converse,Wyoming,1002,17.00,56009,WY,County,13822,7249.31,122.99,122.99,7249.31,20
960302,2021-05-13,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21
1068928,2021-05-13,Niobrara,Wyoming,161,2.00,56027,WY,County,2356,6833.62,84.89,84.89,6833.62,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
633081,2021-05-13,Lowndes,Alabama,1390,53.00,1085,AL,County,9726,14291.59,544.93,544.93,14291.59,3,1
809265,2021-05-13,Hale,Alabama,2232,77.00,1065,AL,County,14651,15234.45,525.56,525.56,15234.45,1,2
269127,2021-05-13,Walker,Alabama,7246,277.00,1127,AL,County,63521,11407.25,436.08,436.08,11407.25,27,3
559632,2021-05-13,Clay,Alabama,1568,57.00,1027,AL,County,13235,11847.37,430.68,430.68,11847.37,19,4
682758,2021-05-13,Greene,Alabama,925,34.00,1063,AL,County,8111,11404.27,419.18,419.18,11404.27,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681105,2021-05-13,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
936348,2021-05-13,Uinta,Wyoming,2237,13.00,56041,WY,County,20226,11060.02,64.27,64.27,11060.02,5,20
960302,2021-05-13,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21,21
267864,2021-05-13,Teton,Wyoming,3762,9.00,56039,WY,County,23464,16033.07,38.36,38.36,16033.07,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4558.79,19,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4558.79,19,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4558.79,19,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4558.79,19,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4558.79,19,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255754,2021-05-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1255755,2021-05-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1255756,2021-05-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1255757,2021-05-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
805909,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15234.45,2,1,1.00
805910,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15234.45,2,1,0.00
805911,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15234.45,2,1,0.00
805912,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15234.45,2,1,0.00
805913,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,525.56,15234.45,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976386,2021-05-09,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5696.20,6,23,0.00
976387,2021-05-10,Crook,Wyoming,431,12.00,56011,WY,County,7584,5683.02,158.23,158.23,5696.20,6,23,2.00
976388,2021-05-11,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,1.00
976389,2021-05-12,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
630545,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14291.59,1,3,1.00,0.00
630546,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14291.59,1,3,0.00,0.00
630547,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14291.59,1,3,0.00,0.00
630548,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14291.59,1,3,0.00,0.00
630549,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14291.59,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628470,2021-05-09,Albany,Wyoming,4329,12.00,56001,WY,County,38880,11134.26,30.86,30.86,11332.30,23,4,0.00,0.00
628471,2021-05-10,Albany,Wyoming,4365,12.00,56001,WY,County,38880,11226.85,30.86,30.86,11332.30,23,4,36.00,0.00
628472,2021-05-11,Albany,Wyoming,4373,12.00,56001,WY,County,38880,11247.43,30.86,30.86,11332.30,23,4,8.00,0.00
628473,2021-05-12,Albany,Wyoming,4388,12.00,56001,WY,County,38880,11286.01,30.86,30.86,11332.30,23,4,15.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-13') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
321299,2021-05-13,Imperial,California,28443,725.00,6025,CA,County,181215,15695.72,400.08,400.08,15695.72,1,2,30.00,3.00
2377,2021-05-13,Los Angeles,California,1237120,24067.00,6037,CA,County,10039107,12323.01,239.73,239.73,12323.01,2,5,213.00,19.00
159935,2021-05-13,San Bernardino,California,296839,4715.00,6071,CA,County,2180085,13615.94,216.28,216.28,13615.94,3,4,57.00,8.00
562904,2021-05-13,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
71493,2021-05-13,Stanislaus,California,62142,1061.00,6099,CA,County,550660,11285.00,192.68,192.68,11285.00,5,9,40.00,1.00
33117,2021-05-13,Riverside,California,299565,4594.00,6065,CA,County,2470546,12125.46,185.95,185.95,12125.46,6,7,110.00,1.00
56875,2021-05-13,San Joaquin,California,73267,1391.00,6077,CA,County,762148,9613.22,182.51,182.51,9613.22,7,18,47.00,0.00
88650,2021-05-13,Tulare,California,50001,839.00,6107,CA,County,466195,10725.34,179.97,179.97,10725.34,8,10,347.00,0.00
32251,2021-05-13,Fresno,California,101838,1688.00,6019,CA,County,999101,10192.96,168.95,168.95,10192.96,9,14,46.00,2.00
510089,2021-05-13,Merced,California,31914,461.00,6047,CA,County,277680,11493.09,166.02,166.02,11493.09,10,8,16.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1192192,2021-05-13,Lassen,California,5701,24.00,6035,CA,County,30573,18647.17,78.50,78.50,18647.17,34,1,0.00,0.00
321299,2021-05-13,Imperial,California,28443,725.00,6025,CA,County,181215,15695.72,400.08,400.08,15695.72,1,2,30.00,3.00
682700,2021-05-13,Kings,California,22999,246.00,6031,CA,County,152940,15037.92,160.85,160.85,15037.92,11,3,10.00,0.00
159935,2021-05-13,San Bernardino,California,296839,4715.00,6071,CA,County,2180085,13615.94,216.28,216.28,13615.94,3,4,57.00,8.00
2377,2021-05-13,Los Angeles,California,1237120,24067.00,6037,CA,County,10039107,12323.01,239.73,239.73,12323.01,2,5,213.00,19.00
201105,2021-05-13,Kern,California,109177,1374.00,6029,CA,County,900202,12128.06,152.63,152.63,12128.06,14,6,38.00,1.00
33117,2021-05-13,Riverside,California,299565,4594.00,6065,CA,County,2470546,12125.46,185.95,185.95,12125.46,6,7,110.00,1.00
510089,2021-05-13,Merced,California,31914,461.00,6047,CA,County,277680,11493.09,166.02,166.02,11493.09,10,8,16.00,1.00
71493,2021-05-13,Stanislaus,California,62142,1061.00,6099,CA,County,550660,11285.00,192.68,192.68,11285.00,5,9,40.00,1.00
88650,2021-05-13,Tulare,California,50001,839.00,6107,CA,County,466195,10725.34,179.97,179.97,10725.34,8,10,347.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
321299,2021-05-13,Imperial,California,28443,725.00,6025,CA,County,181215,15695.72,400.08,400.08,15695.72,1,2,30.00,3.00
2377,2021-05-13,Los Angeles,California,1237120,24067.00,6037,CA,County,10039107,12323.01,239.73,239.73,12323.01,2,5,213.00,19.00
159935,2021-05-13,San Bernardino,California,296839,4715.00,6071,CA,County,2180085,13615.94,216.28,216.28,13615.94,3,4,57.00,8.00
562904,2021-05-13,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
71493,2021-05-13,Stanislaus,California,62142,1061.00,6099,CA,County,550660,11285.00,192.68,192.68,11285.00,5,9,40.00,1.00
33117,2021-05-13,Riverside,California,299565,4594.00,6065,CA,County,2470546,12125.46,185.95,185.95,12125.46,6,7,110.00,1.00
56875,2021-05-13,San Joaquin,California,73267,1391.00,6077,CA,County,762148,9613.22,182.51,182.51,9613.22,7,18,47.00,0.00
88650,2021-05-13,Tulare,California,50001,839.00,6107,CA,County,466195,10725.34,179.97,179.97,10725.34,8,10,347.00,0.00
32251,2021-05-13,Fresno,California,101838,1688.00,6019,CA,County,999101,10192.96,168.95,168.95,10192.96,9,14,46.00,2.00
510089,2021-05-13,Merced,California,31914,461.00,6047,CA,County,277680,11493.09,166.02,166.02,11493.09,10,8,16.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,400.08,15695.72,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4212,05/09/21,Lassen,5698,24.00,18637.36,78.50,78.50,18647.17,34,1,0.00,0.00
4213,05/10/21,Lassen,5698,24.00,18637.36,78.50,78.50,18647.17,34,1,0.00,0.00
4214,05/11/21,Lassen,5701,24.00,18647.17,78.50,78.50,18647.17,34,1,3.00,0.00
4215,05/12/21,Lassen,5701,24.00,18647.17,78.50,78.50,18647.17,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,400.08,15695.72,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,400.08,15695.72,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4298,05/09/21,Merced,31743,458.00,11431.50,164.94,166.02,11493.09,10,8,0.00,0.00
4299,05/10/21,Merced,31821,459.00,11459.59,165.30,166.02,11493.09,10,8,78.00,1.00
4300,05/11/21,Merced,31859,460.00,11473.28,165.66,166.02,11493.09,10,8,38.00,1.00
4301,05/12/21,Merced,31898,460.00,11487.32,165.66,166.02,11493.09,10,8,39.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)